# Fix binning, padding, and cropping of various datasets

You should not need to interact with this code directly, but in case you need to deal with images with unusual voxel sizes, this code will help you rebin, pad, and crop them as necessary. This may become relevant for trying to use other labs' data in the Segmentation Net.

The original images 15-19 had unusual voxel sizes, with initial labels generated post-cropping. To change the crop size, we need to rebin the images and then un-crop the labels.

Additionally, original images 23 and 29 were transposed. They need to be padded in the y-dimension to update the crop size to a larger value.

In [ ]:
crop_dict_orig = Dict(
    15 => [100:419, 210:353, 85:224],
    16 => [51:370, 108:251, 55:194],
    17 => [30:349, 220:363, 21:160],
    18 => [212:355, 189:508, 56:195],
    19 => [175:318, 125:444, 34:173],
    20 => [195:338, 135:454, 1:77],
    21 => [175:318, 135:454, 1:77],
    22 => [185:328, 135:454, 1:77],
    23 => [100:195, 1:210, 32:82],
    24 => [61:270, 70:165, 21:71],
    25 => [61:270, 60:155, 34:84],
    26 => [51:260, 40:135, 28:78],
    27 => [51:260, 40:135, 36:86],
    28 => [51:260, 55:150, 40:90],
    29 => [105:200, 1:210, 40:90]
); # original crop size parameters for all datasets

In [ ]:
new_crop_dict = Dict(); # new crop sizes

In [ ]:
input_label_dir = "label_cropped" # subpath containing labels
input_raw_dir = "img_uncropped" # subpath containing images

output_img_dir = joinpath(rootpath, "img_binned_uncropped")
output_label_dir = joinpath(rootpath, "NEW_label_uncropped");

### Bin images 15-19 and uncrop their labels

In [ ]:
for i in 15:19
    mhd_img = MHD(joinpath(rootpath, input_raw_dir, "$(i)_img.mhd"))
    img = MHDIO.read_img(mhd_img) # UNBINNED, UNCROPPED image

    nrrd_label = NRRD(joinpath(rootpath, input_label_dir, "$(i)_label.nrrd"))
    label = NRRDIO.read_img(nrrd_label) # BINNED, CROPPED label

    # only the LABEL will be (un-)transposed
    if i in 18:19
        label = permutedims(label, [2,1,3])
    end
    
    bin_scale = [1.5,1.5,2.7]
    crop = crop_dict_orig[i]


    # starting points of crop on the new image
    new_crop = [1,1,1]
    for j=1:3
        is_int = false
        while !is_int
            resampled_crop = (crop[j][1]-new_crop[j])/bin_scale[j]+1 # starting point of crop
            if resampled_crop == floor(resampled_crop) # check if int
                is_int = true
            else
                new_crop[j] += 1
            end
        end
    end

    new_crop_dict[i] = new_crop

    img = img[new_crop[1]:end, new_crop[2]:end, new_crop[3]:end]; # crop image to ensure integer binning offset

    resampled_img = resample_img(img, bin_scale) # resample image to match label size

    label_uncropped = zeros(size(resampled_img))

    # Extract start and end indices from crop
    crop_start = [first(crop[j]) for j in 1:3]
    crop_end = [last(crop[j]) for j in 1:3]

    # Calculate the corresponding uncropped region in the resampled image
    label_uncropped_start = Int.((crop_start .- new_crop) ./ bin_scale .+ 1)
    label_uncropped_end = label_uncropped_start .+ Int.(floor.((crop_end .- crop_start .+ 1) ./ bin_scale)) .- 1

    # Initialize the uncropped label matrix with the size of the resampled image
    label_uncropped = zeros(size(resampled_img))

    # Place the cropped label into the correct uncropped position
    label_uncropped[label_uncropped_start[1]:label_uncropped_end[1], 
                label_uncropped_start[2]:label_uncropped_end[2], 
                label_uncropped_start[3]:label_uncropped_end[3]] .= label

    # Save the resampled image and the uncropped label
    MHDIO.write_raw(joinpath(output_img_dir, "$(i)_img.raw"), resampled_img)
    MHDIO.write_MHD_spec(joinpath(output_img_dir, "$(i)_img.mhd"), 0.54, 0.54, size(resampled_img)..., "$(i)_img.raw")
    NRRDIO.write_nrrd(joinpath(output_label_dir, "$(i)_label.nrrd"), label_uncropped, (0.54, 0.54, 0.54))
end


In [ ]:
resampled_crop_dict = Dict()

for i=15:19
    crop = crop_dict_orig[i]
    new_crop = new_crop_dict[i]

    if i in 18:19
        crop = [crop[2], crop[1], crop[3]]
    end
    
    bin_scale = [1.5,1.5,2.7]

    crop_start = [first(crop[j]) for j in 1:3]
    crop_end = [last(crop[j]) for j in 1:3]

    # Calculate the corresponding uncropped region in the resampled image
    label_uncropped_start = Int.((crop_start .- new_crop) ./ bin_scale .+ 1)
    label_uncropped_end = label_uncropped_start .+ Int.(floor.((crop_end .- crop_start .+ 1) ./ bin_scale)) .- 1

    resampled_crop_dict[i] = [label_uncropped_start[j]:label_uncropped_end[j] for j in 1:3]
end

### Pad images 23 and 29

In [ ]:
function pad_to_square(image::Array{T, 3}) where T
    size_x, size_y, size_z = size(image)
    
    # Determine the median value of the entire image and cast it to the original data type
    median_value = T(floor(median(image)))

    # Initialize padding amounts
    padding_front = (0, 0, 0)
    padded_image = image

    if size_x < size_y
        # Pad along the x-axis
        pad_size = size_y - size_x
        pad_before = div(pad_size, 2)
        pad_after = pad_size - pad_before
        padded_image = fill(median_value, size_y, size_y, size_z)
        padded_image[pad_before+1:pad_before+size_x, :, :] .= image
        padding_front = (pad_before, 0, 0)
    elseif size_y < size_x
        # Pad along the y-axis
        pad_size = size_x - size_y
        pad_before = div(pad_size, 2)
        pad_after = pad_size - pad_before
        padded_image = fill(median_value, size_x, size_x, size_z)
        padded_image[:, pad_before+1:pad_before+size_y, :] .= image
        padding_front = (0, pad_before, 0)
    end

    return padded_image, padding_front
end


In [ ]:
for i in [23, 29]
    img = MHDIO.read_img(MHD(joinpath(rootpath, input_raw_dir, "$(i)_img.mhd")))
    label = NRRDIO.read_img(NRRD(joinpath(rootpath, input_label_dir, "$(i)_label.nrrd")))

    img, pad1 = pad_to_square(img)
    label, pad2 = pad_to_square(label)

    println("$(i): $(pad1)")
    println("$(i): $(pad2)")


    MHDIO.write_raw(joinpath(output_img_dir, "$(i)_img.raw"), img)
    MHDIO.write_MHD_spec(joinpath(output_img_dir, "$(i)_img.mhd"), 0.54, 0.54, size(img)..., "$(i)_img.raw")
    NRRDIO.write_nrrd(joinpath(output_label_dir, "$(i)_label.nrrd"), label, (0.54, 0.54, 0.54))
end